# Relatório do Projeto Final

## Machine Learning - 2023.1

### Autores: Marco Moliterno e Renato Falcão

Este relatório tem o objetivo de descrever a aplicação de técnicas de aprendizado de máquina para prever o preço de cartas de Yu-Gi-Oh baseando-se nas múltiplas características das mesmas.

O dataset estudado é uma união da base de dados, disponibilizada no [Kaggle](https://www.kaggle.com/), com os preços médios das cartas, disponibilizada pela API do website [Yu-Gi-Oh Prices](https://yugiohprices.com/).

#### Referências

Base de dados do Kaggle disponível [aqui](https://www.kaggle.com/datasets/thedevastator/yu-gi-oh-dataset?select=yugioh_enriched.csv).

Requisições web realizadas ao iterar sobre os sets de cartas, obtendo os preços das cartas pertencentes aos respectivos sets. Esta API pode ser consumida pelo endpoint `https://yugiohprices.com/api/set_data/{nome_do_set}`. Exemplo de resposta pode ser observado [aqui](https://yugiohprices.com/api/set_data/2013%20Collectible%20Tins%20Wave%201).

## Inicio do projeto

### Importando bibliotecas e carregando base de dados

Para realizar o estudo, estaremos utilizando especialmente as bibliotecas *Pandas*, *MatPlotLib*, *Scikit-Learn* e *Numpy*. Vamos importá-las e carregar a base de dados com o Pandas.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv("dbs/yugi_db_polished.csv", index_col=0)

df.head()

,Name,Rarity,Price,Description,CardType,Attribute,ATK,DEF,LVL,Property,MonsterType,isEffect,Duelist,Ability1,Ability2,isTuner,isPendulum
0,XX-Saber Boggart Knight,Shatterfoil Rare,2.73,When this card is Normal Summoned: You can Spe...,monster,earth,1900.0,1000.0,4.0,NaN,Beast-Warrior,Effect,NaN,NaN,NaN,NaN,NaN
1,Gagaga Cowboy,Shatterfoil Rare,4.68,2 Level 4 monsters\r\n\r\nOnce per turn: You c...,monster,earth,1500.0,2400.0,4.0,NaN,Warrior,Effect,NaN,Xyz,NaN,NaN,NaN
2,Forbidden Chalice,Shatterfoil Rare,2.09,Target 1 face-up monster on the field; until t...,spell,NaN,NaN,NaN,NaN,Quick-Play,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fairy Cheer Girl,Shatterfoil Rare,2.06,2 Level 4 Fairy-Type monsters\r\n\r\nYou can d...,monster,light,1900.0,1500.0,4.0,NaN,Fairy,Effect,NaN,Xyz,NaN,NaN,NaN
4,Exploder Dragon,Shatterfoil Rare,2.08,If this card is destroyed by battle and sent t...,monster,earth,1000.0,0.0,3.0,NaN,Dragon,Effect,NaN,NaN,NaN,NaN,NaN


### Análise Exploratória e Limpeza dos Dados

Com os dados em mãos, podemos começar realizando uma análise exploratória na qual desejamos identificar as características do dataset, observando a estrutura dos dados disponíveis, presença de anomalias, tendências, padrões, distribuições e dados ausentes. O entendimento dos dados é fundamental para se poder tomar decisões com bom embasamento acerca da abordagem que utilizaremos.

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11574 entries, 0 to 11573
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         11574 non-null  object 
 1   Rarity       11574 non-null  object 
 2   Price        11574 non-null  float64
 3   Description  11574 non-null  object 
 4   CardType     11574 non-null  object 
 5   Attribute    7470 non-null   object 
 6   ATK          7580 non-null   float64
 7   DEF          7580 non-null   float64
 8   LVL          7580 non-null   float64
 9   Property     4096 non-null   object 
 10  MonsterType  7478 non-null   object 
 11  isEffect     7478 non-null   object 
 12  Duelist      102 non-null    object 
 13  Ability1     1696 non-null   object 
 14  Ability2     137 non-null    object 
 15  isTuner      453 non-null    object 
 16  isPendulum   277 non-null    object 
dtypes: float64(4), object(13)
memory usage: 1.6+ MB


In [3]:
df.columns

Index(['Name', 'Rarity', 'Price', 'Description', 'CardType', 'Attribute',
       'ATK', 'DEF', 'LVL', 'Property', 'MonsterType', 'isEffect', 'Duelist',
       'Ability1', 'Ability2', 'isTuner', 'isPendulum'],
      dtype='object')

**Name**: O nome da carta. Cada carta de Yu-Gi-Oh tem um nome único que a distingue de outras cartas.

**Rarity**: Refere-se à frequência com que a carta aparece em pacotes de cartas. As cartas raras são mais difíceis de encontrar do que as comuns. Exemplos de raridades incluem "Common", "Rare", "Ultra Rare", "Secret Rare", entre outros.

**Price**: O preço de mercado da carta. Isso pode variar com base em vários fatores, como raridade, demanda, condição da carta e se ela está em circulação.

**Description**: Uma descrição textual da carta que geralmente inclui suas habilidades, efeitos ou lore.

**CardType**: O tipo de carta. As cartas de Yu-Gi-Oh podem ser de vários tipos, como Monstros, Magias, ou Armadilhas.

**Attribute**: Este é um traço específico dos monstros. Existem 6 atributos principais: Luz, Trevas, Fogo, Água, Terra e Vento.

**ATK**: Abreviação de "Attack Points". Este é o poder de ataque de uma carta de monstro. Quando dois monstros batalham, o monstro com o ATK mais alto geralmente destrói o monstro com o ATK mais baixo.

**DEF**: Abreviação de "Defense Points". Este é o poder de defesa de uma carta de monstro. É usado quando o monstro está em posição de defesa.

**LVL**: Abreviação de "Level". Este é o nível de uma carta de monstro. O nível de um monstro geralmente determina seu poder, e também é usado para o mecanismo de invocação de tributo.

**Property**: Este é um atributo de cartas de Magia e Armadilha. As propriedades de cartas de Magia incluem "Normal", "Contínua", "Equipamento", "Campo", "Rápida", etc. As cartas de Armadilha têm propriedades "Normal", "Contínua" e "Contra".

**MonsterType**: Isto é específico para cartas de monstros e refere-se à sua classificação. Exemplos incluem "Guerreiro", "Besta", "Dragão", "Demoníaco", etc.

**isEffect**: Um indicador de se o monstro é uma carta de Monstro de Efeito. Estes monstros têm habilidades especiais que são ativadas sob certas condições.

**Duelist**: O duelista que utiliza a carta. Isso pode ser relevante para a história do jogo ou para certas habilidades de cartas.

**Ability1** e **Ability2**: Estes se referem a habilidades especiais ou efeitos que a carta pode ter. Algumas cartas podem ter várias habilidades.

**isTuner**: Um indicador de se a carta é um Monstro Tuner. Estes monstros são usados para Invocações Sincro.

**isPendulum**: Um indicador de se a carta é um Monstro de Pêndulo. Estes monstros podem ser invocados usando a mecânica de Invocação por Pêndulo. Além disso, eles têm escalas de Pêndulo que permitem a Invocação de vários monstros.

É possível notar que diversas colunas têm uma grande quantidade de dados faltantes. No entanto, esses dados "faltantes" significam, na verdade, se as cartas possuem certos atributos ou não. Para analisar as características individuais de cada tipo de carta ("CardType"), podemos analisar um recorte de cada um dos tipos:

In [4]:
text_card_caracteristics = ["Name", "Description"]
categoric_card_caracteristics = ["CardType", "Rarity", "Attribute", "Property", "MonsterType", "Duelist", "Ability1", "Ability2"]
boolean_card_caracteristics = ["isEffect", "isTuner", "isPendulum"]
numeric_card_caracteristics = ["ATK", "DEF", "LVL", "Price"]

card_types = df["CardType"].unique()

for card_type in card_types:
    print(f"Card Type: {card_type}")
    for caracteristica in categoric_card_caracteristics[1:]:
        recorte = df[df["CardType"] == card_type]
        print(f"{caracteristica}: {recorte[caracteristica].unique()}")
    for caracteristica in boolean_card_caracteristics:
        recorte = df[df["CardType"] == card_type]
        print(f"{caracteristica}: {recorte[caracteristica].unique()}")
    print("\n")

Card Type: monster
Rarity: ['Shatterfoil Rare' 'Mosaic Rare' 'Common' 'Platinum Secret Rare'
 'Platinum Rare' 'Short Print' 'Starfoil Rare' 'Gold Secret Rare' 'Rare'
 'Premium Gold Rare' 'Gold Rare' "Collector's Rare" 'Collectors Rare'
 'Duel Terminal Ultra Parallel Rare' 'Prismatic Secret Rare'
 'Duel Terminal Normal Parallel Rare' 'Super Rare'
 'Duel Terminal Super Parallel Rare' 'Ultimate Rare' 'Secret Rare'
 'Ultra Rare' '10000 Secret Rare']
Attribute: ['earth' 'light' 'water' 'dark' 'wind' 'divine' 'fire' nan '?']
Property: [nan]
MonsterType: ['Beast-Warrior' 'Warrior' 'Fairy' 'Dragon' 'Aqua' 'Zombie' 'Rock' 'Beast'
 'Insect' 'Sea Serpent' 'Fiend' 'Reptile' 'Plant' 'Thunder' 'Spellcaster'
 'Machine' 'Winged Beast' 'Divine-Beast' 'Dinosaur' 'Pyro' 'Fish' 'Normal'
 'Psychic' 'Cyberse' 'Wyrm' 'Token']
Duelist: [nan]
Ability1: [nan 'Xyz' 'Fusion' 'Synchro' 'Ritual' 'Toon' 'Link' 'Union' 'Gemini']
Ability2: [nan 'Flip' 'Spirit']
isEffect: ['Effect' 'Normal']
isTuner: [nan 'Tuner']
isPe

Desta forma, é possível inferir a seguinte relação dos tipos de cartas para as características que podem possuir:

- **Monster**: Possui *Attribute*, *ATK*, *DEF*, *LVL*, *MonsterType*, *isEffect*, *Ability1*, *Ability2*, *isTuner* e *isPendulum*;
- **Spell**: Possui apenas *Property*;
- **Trap**: Possui apenas *Property*;
- **Skill**: Possui *Property* e *Duelist*;

#### Cartas do tipo Monster

Podemos perceber que existem cartas do tipo **Monster** cujo *Attribute* é "NaN" e "?", ou seja, são valores nulos. Observando esses valores, obtemos o seguinte:

In [5]:
anomalias_nan = df[df["CardType"] == "monster"][df["Attribute"].isna() == True]
anomalias_nan

C:\Users\renat\AppData\Local\Temp\ipykernel_7560\321881468.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  anomalias_nan = df[df["CardType"] == "monster"][df["Attribute"].isna() == True]


,Name,Rarity,Price,Description,CardType,Attribute,ATK,DEF,LVL,Property,MonsterType,isEffect,Duelist,Ability1,Ability2,isTuner,isPendulum
209,Token,Common,4.23,This card can be used as any Token.,monster,NaN,0.0,0.0,0.0,NaN,Normal,Normal,NaN,NaN,NaN,NaN,NaN
4195,Ecclesia the Exiled,Common,1.21,This card can be used as any Token or Counter....,monster,NaN,0.0,0.0,0.0,NaN,Token,Normal,NaN,NaN,NaN,NaN,NaN
4206,Albaz the Shrouded,Common,1.21,This card can be used as any Token or Counter....,monster,NaN,0.0,0.0,0.0,NaN,Token,Normal,NaN,NaN,NaN,NaN,NaN
4209,Aluber the Dogmatic,Common,1.25,This card can be used as any Token or Counter....,monster,NaN,0.0,0.0,0.0,NaN,Token,Normal,NaN,NaN,NaN,NaN,NaN
4229,The Virtuous Vestals,Common,1.21,This card can be used as any Token or Counter....,monster,NaN,0.0,0.0,0.0,NaN,Token,Normal,NaN,NaN,NaN,NaN,NaN
4233,Tri-Brigade,Common,1.22,This card can be used as any Token or Counter....,monster,NaN,0.0,0.0,0.0,NaN,Token,Normal,NaN,NaN,NaN,NaN,NaN
9369,Jesse Anderson - Bonder with the Crystal Beasts,Super Rare,1.25,This card can be used as any Token or Counter....,monster,NaN,0.0,0.0,0.0,NaN,Normal,Normal,NaN,NaN,NaN,NaN,NaN
9370,Jesse and Ruby - Unleashing the Legend,Super Rare,1.25,This card can be used as any Token or Counter....,monster,NaN,0.0,0.0,0.0,NaN,Normal,Normal,NaN,NaN,NaN,NaN,NaN


In [6]:
anomalias_interrogacao = df[df["CardType"] == "monster"][df["Attribute"] == "?"]
anomalias_interrogacao

C:\Users\renat\AppData\Local\Temp\ipykernel_7560\4057386446.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  anomalias_interrogacao = df[df["CardType"] == "monster"][df["Attribute"] == "?"]


,Name,Rarity,Price,Description,CardType,Attribute,ATK,DEF,LVL,Property,MonsterType,isEffect,Duelist,Ability1,Ability2,isTuner,isPendulum
7801,Crystal Beast Token,Common,1.18,"This card can be used as a ""Crystal Beast Toke...",monster,?,0.0,0.0,0.0,NaN,Normal,Normal,NaN,NaN,NaN,NaN,NaN
10216,Duel Dragon Token,Super Rare,1.84,"This card can be used as a ""Duel Dragon Token""...",monster,?,0.0,0.0,0.0,NaN,Normal,Normal,NaN,NaN,NaN,NaN,NaN
10464,Option Token,Super Rare,1.48,"This card can be used as an ""Option Token"".\r\...",monster,?,0.0,0.0,0.0,NaN,Normal,Normal,NaN,NaN,NaN,NaN,NaN


Essas cartas na verdade são **Tokens** do jogo, que são apenas representações de **Monstros**. Desta forma, não vamos considerá-las para a análise.

In [7]:
df.drop(anomalias_nan.index, inplace=True)
df.drop(anomalias_interrogacao.index, inplace=True)

Outro detalhe que chama a atenção são as colunas **isEffect**, **isTuner** e **isPendulum**. Essas colunas indicam um propriedade específica das cartas e podem ser transformadas em variáveis booleanas: *0* para *Não* e *1* para *Sim*.

In [8]:
df["isEffect"].replace(np.nan, 0, inplace=True)
df["isEffect"].replace("Normal", 0, inplace=True)
df["isEffect"].replace("Effect", 1, inplace=True)

df["isTuner"].replace(np.nan, 0, inplace=True)
df["isTuner"].replace("Tuner", 1, inplace=True)

df["isPendulum"].replace(np.nan, 0, inplace=True)
df["isPendulum"].replace("Pendulum", 1, inplace=True)

Ainda, é possível notar que existem muitos valores "NaN", que é uma forma de identificar valores *nulos*. No entanto, os valores nulos são, neste caso, informações úteis também, já que indicam quando as cartas não possuem certos atributos. Desta forma, vamos substituir todos os valores nulos por categorias que, por sua vez, representem a não existência de informação.

In [9]:
for caracteristica in categoric_card_caracteristics:
    df[caracteristica].replace(np.nan, "None", inplace=True)

for caracteristica in numeric_card_caracteristics:
    df[caracteristica].replace(np.nan, 0, inplace=True)

Por fim, podemos definir os tipos das variáveis, identificando-as como *texto*, *categóricas* ou *numéricas* .

In [10]:
for caracteristica in text_card_caracteristics:
    df[caracteristica] = df[caracteristica].astype("string")

for caracteristica in boolean_card_caracteristics:
    df[caracteristica] = df[caracteristica].astype("int64")

for caracteristica in categoric_card_caracteristics:
    df[caracteristica] = df[caracteristica].astype("category")

for caracteristica in numeric_card_caracteristics:
    df[caracteristica] = df[caracteristica].astype("float64")

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11563 entries, 0 to 11573
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Name         11563 non-null  string  
 1   Rarity       11563 non-null  category
 2   Price        11563 non-null  float64 
 3   Description  11563 non-null  string  
 4   CardType     11563 non-null  category
 5   Attribute    11563 non-null  category
 6   ATK          11563 non-null  float64 
 7   DEF          11563 non-null  float64 
 8   LVL          11563 non-null  float64 
 9   Property     11563 non-null  category
 10  MonsterType  11563 non-null  category
 11  isEffect     11563 non-null  int64   
 12  Duelist      11563 non-null  category
 13  Ability1     11563 non-null  category
 14  Ability2     11563 non-null  category
 15  isTuner      11563 non-null  int64   
 16  isPendulum   11563 non-null  int64   
dtypes: category(8), float64(4), int64(3), string(2)
memory usage: 997.9 K

### Pré-processamento dos dados

Agora que temos um dataset limpo, a ideia é realizar um pré-processamento desses dados. Nesta etapa, os dados são transformados em uma forma adequada para serem utilizados pelos modelos de Machine Learning. Isso pode envolver a normalização ou padronização das variáveis, a codificação de variáveis categóricas, a redução de dimensionalidade e outras técnicas de preparação dos dados.

In [12]:
model_data = df.copy()

#### Tratando variáveis nominais



In [13]:
dummies_data = pd.get_dummies(model_data[categoric_card_caracteristics], drop_first=True)
model_data = model_data[text_card_caracteristics].join(model_data[numeric_card_caracteristics]).join(model_data[boolean_card_caracteristics]).join(dummies_data)


In [14]:
model_data.head()

,Name,Description,ATK,DEF,LVL,Price,isEffect,isTuner,isPendulum,CardType_skill,...,Ability1_Gemini,Ability1_Link,Ability1_None,Ability1_Ritual,Ability1_Synchro,Ability1_Toon,Ability1_Union,Ability1_Xyz,Ability2_None,Ability2_Spirit
0,XX-Saber Boggart Knight,When this card is Normal Summoned: You can Spe...,1900.0,1000.0,4.0,2.73,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
1,Gagaga Cowboy,2 Level 4 monsters Once per turn: You can d...,1500.0,2400.0,4.0,4.68,1,0,0,0,...,0,0,0,0,0,0,0,1,1,0
2,Forbidden Chalice,Target 1 face-up monster on the field; until t...,0.0,0.0,0.0,2.09,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
3,Fairy Cheer Girl,2 Level 4 Fairy-Type monsters You can detac...,1900.0,1500.0,4.0,2.06,1,0,0,0,...,0,0,0,0,0,0,0,1,1,0
4,Exploder Dragon,If this card is destroyed by battle and sent t...,1000.0,0.0,3.0,2.08,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


### Text Embedding

A informação da descrição ("Description") das cartas, apesar de ser um texto complexo e não apenas uma variável categórica ou numérica, ainda pode ser transformado em informação útil para o modelo. A solução é utilizar um modelo pré-treinado do Keras para obter um vetor numérico a partir das palavras da descrição. Com este vetor, é possível realizar uma decomposição conhecida com *Principal Component Analysis* (ou *PCA*), reduzindo este vetor à apenas 5 números, que serão passados como parâmetros para o modelo de regressão.

In [15]:
import tensorflow_hub as hub
from sklearn.decomposition import PCA

text_emb_model = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim128/2")

df_emb = df.copy()

In [16]:
df_emb["Description"] = df_emb["Description"].astype("object")
embeddings = text_emb_model(df_emb["Description"])
embeddings.shape

TensorShape([11563, 128])

In [17]:
pca_model = PCA(n_components=10)
pca_DF = pca_model.fit_transform(embeddings)
pca_DF

array([[ 0.18863321,  0.24797628,  0.17773357, ...,  0.02231981,
        -0.07409769,  0.19903699],
       [ 0.19556614, -0.0337155 , -0.38718466, ..., -0.0450269 ,
        -0.03139706, -0.32055491],
       [ 0.58689258, -0.31326951, -0.02875208, ..., -0.06833587,
        -0.12442974, -0.10872888],
       ...,
       [-0.51233992, -0.10810647,  0.15313301, ..., -0.10140878,
        -0.05447631, -0.01016006],
       [-0.53554939, -0.47601408,  0.27209389, ...,  0.1433081 ,
        -0.23961796, -0.13748154],
       [-0.03183401, -0.36062723,  0.45437434, ..., -0.24520914,
         0.11858419,  0.04522151]])

In [18]:
df_pca = pd.DataFrame(pca_DF, columns=['pca1', 'pca2','pca3','pca4','pca5', 'pca6','pca7','pca8','pca9','pca10'])
df_pca.head()

,pca1,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,pca10
0,0.188633,0.247976,0.177734,0.403867,-0.124979,-0.264077,-0.023245,0.022320,-0.074098,0.199037
1,0.195566,-0.033715,-0.387185,0.149372,0.255015,-0.140675,-0.006841,-0.045027,-0.031397,-0.320555
2,0.586893,-0.313270,-0.028752,0.187890,0.207091,-0.112275,-0.374577,-0.068336,-0.124430,-0.108729
3,0.240599,0.365433,-0.041543,0.027833,0.274027,-0.227894,-0.093132,0.363715,0.317084,-0.152312
4,0.336299,-0.863256,0.091189,-0.370700,0.017590,-0.276843,0.389737,-0.005346,-0.036967,0.045706


In [19]:
model_data.reset_index(drop=True, inplace=True)
model_data = model_data.join(df_pca)
model_data

,Name,Description,ATK,DEF,LVL,Price,isEffect,isTuner,isPendulum,CardType_skill,...,pca1,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,pca10
0,XX-Saber Boggart Knight,When this card is Normal Summoned: You can Spe...,1900.0,1000.0,4.0,2.73,1,0,0,0,...,0.188633,0.247976,0.177734,0.403867,-0.124979,-0.264077,-0.023245,0.022320,-0.074098,0.199037
1,Gagaga Cowboy,2 Level 4 monsters Once per turn: You can d...,1500.0,2400.0,4.0,4.68,1,0,0,0,...,0.195566,-0.033715,-0.387185,0.149372,0.255015,-0.140675,-0.006841,-0.045027,-0.031397,-0.320555
2,Forbidden Chalice,Target 1 face-up monster on the field; until t...,0.0,0.0,0.0,2.09,0,0,0,0,...,0.586893,-0.313270,-0.028752,0.187890,0.207091,-0.112275,-0.374577,-0.068336,-0.124430,-0.108729
3,Fairy Cheer Girl,2 Level 4 Fairy-Type monsters You can detac...,1900.0,1500.0,4.0,2.06,1,0,0,0,...,0.240599,0.365433,-0.041543,0.027833,0.274027,-0.227894,-0.093132,0.363715,0.317084,-0.152312
4,Exploder Dragon,If this card is destroyed by battle and sent t...,1000.0,0.0,3.0,2.08,1,0,0,0,...,0.336299,-0.863256,0.091189,-0.370700,0.017590,-0.276843,0.389737,-0.005346,-0.036967,0.045706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11558,Herald of Mirage Lights,"2 monsters with the same Type and Attribute, e...",600.0,0.0,0.0,1.55,1,0,0,0,...,-0.589056,0.216736,0.329789,-0.301808,-0.248366,0.101903,-0.036688,0.139482,-0.107377,0.182720
11559,Graveyard of Wandering Souls,If this card is already face-up on the field: ...,0.0,0.0,0.0,1.39,0,0,0,0,...,-0.167996,-0.135148,0.082802,0.198532,-0.091635,0.030850,-0.007164,0.239857,-0.212095,0.079176
11560,Lib the World Key Blademaster,2 monsters Can only be Link Summoned while ...,2000.0,0.0,0.0,3.47,1,0,0,0,...,-0.512340,-0.108106,0.153133,0.076935,-0.046781,0.234145,-0.186295,-0.101409,-0.054476,-0.010160
11561,Kingyo Sukui,You can target 1 monster in your opponent's GY...,0.0,0.0,0.0,1.22,0,0,0,0,...,-0.535549,-0.476014,0.272094,-0.110142,-0.133400,0.013295,-0.258434,0.143308,-0.239618,-0.137482


### Divisão dos dados

Antes de prosseguir para a seleção de um modelo e treinamento, é necessário dividir os dados em conjuntos de treinamento, validação e teste. O conjunto de treinamento é utilizado para treinar o modelo, o conjunto de validação é usado para ajustar os parâmetros do modelo e o conjunto de teste é utilizado para avaliar o desempenho final do modelo. Esta divisão é importante para evitar que a avaliação do modelo seja enviesada pelo resultado final, de forma a "overfittar" para os dados de teste. 

In [21]:
X = model_data.drop(columns=["Name", "Description", "Price"]).copy().values
y = model_data["Price"].copy().values

In [22]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=42,
)

In [23]:
X.shape, Xtrain.shape, Xtest.shape

((11563, 130), (8672, 130), (2891, 130))

In [24]:
y.shape, ytrain.shape, ytest.shape

((11563,), (8672,), (2891,))

### Seleção e Treinamento do modelo

O modelo selecionado é o ... e será treinado usando o conjunto de treinamento. O treinamento envolve alimentar os dados ao modelo e ajustar seus parâmetros com o objetivo de aprender os padrões presentes nos dados.

#### Regressão linear sem feature engineering

In [25]:
from sklearn.linear_model import LinearRegression

In [26]:
model = LinearRegression()

model.fit(Xtrain, ytrain)

LinearRegression()

In [27]:
ypred = model.predict(Xtest)

### Avaliação do modelo

O desempenho do modelo é avaliado usando o conjunto de validação. Métricas como acurácia, precisão, recall, F1-score e curvas de aprendizado são utilizadas para avaliar o quão bem o modelo está generalizando os dados.

In [28]:
from sklearn.metrics import mean_squared_error

RMSE = np.sqrt(mean_squared_error(ytest, ypred))

In [29]:
RMSE

54.55979145335049

### Ajuste do modelo

Com base na avaliação do modelo, é possível ajustar os parâmetros, selecionar diferentes algoritmos de treinamento ou fazer outras modificações para melhorar o desempenho do modelo.

### Teste do modelo

O modelo ajustado é então testado usando o conjunto de teste, que contém dados não vistos anteriormente. Isso permite avaliar como o modelo se comporta em situações reais e verificar sua capacidade de generalização.